In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import calendar
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
from wordcloud import WordCloud, STOPWORDS
import folium
from folium import plugins

# 1. Merge datasets

In [ ]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/NUSE 934 611(M) 2017-2018.dsv'
data2018=pd.read_csv(data_location,delimiter="|")

In [ ]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/NUSE 934-611-611M ENERO2019.csv'
data2019=pd.read_csv(data_location,delimiter=";")

In [ ]:
frames = [data2018, data2019]
data = pd.concat(frames)
merged_nuse = data.loc[data['TIPO_DETALLE'] == '934 - RIÑA']
merged_nuse.reset_index(inplace=True)
merged_nuse.drop(columns=['index'], inplace=True)

In [ ]:
merged_nuse.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/merged_nuse.csv',index=None)

# 2. Rebuild missing data

In [36]:
localidadCodDictionaryNuse = {1:'USAQUEN',
                              2:'CHAPINERO',
                              3:'SANTA FE',
                              4:'SAN CRISTOBAL',
                              5:'USME',
                              6:'TUNJUELITO',
                              7:'BOSA',
                              8:'KENNEDY',
                              9:'FONTIBON',
                              10:'ENGATIVA',
                              11:'SUBA',
                              12:'BARRIOS UNIDOS',
                              13:'TEUSAQUILLO',
                              14:'LOS MARTIRES',
                              15:'ANTONIO NARIÑO',
                              16:'PUENTE ARANDA',
                              17:'CANDELARIA',
                              18:'RAFAEL URIBE URIBE',
                              19:'CIUDAD BOLIVAR',
                              20:'SUMAPAZ',
                              99:'SIN LOCALIZACION'}

## Methods to rebuild

In [41]:
import import_ipynb
import ws_address
from selenium.common.exceptions import TimeoutException
import re
import unidecode

In [ ]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [ ]:
tags = ["Dirección ingresada: ","Dirección encontrada: ","Tipo dirección: ","Código postal: ","Sector catastral: ",
        "UPZ: ","Localidad: ","Latitud: ","Longitud: ","CHIP: "]
def parse_address_ws(ws_result):
    location = {}
    for idx in range(len(tags)-1):
        location[tags[idx].replace(': ','')] = find_between(ws_result,tags[idx],tags[idx+1])
    return location

In [ ]:
def assign_upz(original_df,index,UPZ_ws_field):
    original_df.at[index,'COD_UPZ'] = find_between(UPZ_ws_field, '(', ')')
    original_df.at[index,'UPZ'] = find_between(UPZ_ws_field, '', ' (')

In [ ]:
def get_cod_localidad(localidad_name):
    return (localidadCodDictionaryNuse.keys())[list(localidadCodDictionaryNuse.values()).index(localidad_name)]

In [43]:
def rebuild_location_in_nuse(original_df, index, driver):
    address = original_df.at[index,'STR_DIRECCION_INCIDENTE']
    print(address)
    result_ws = ws_address.web_scrap_address(driver,address)
    ws_address.delete_address(driver,address)
    print(result_ws)

    if result_ws != "Not found":
        parsed_result = parse_address_ws(result_ws)
        print(parsed_result)
        original_df.at[index,'LATITUD'] = float(parsed_result['Latitud'])
        original_df.at[index,'LONGITUD'] = float(parsed_result['Longitud'])
        original_df.at[index,'LOCALIDAD'] = int(unidecode.unidecode(parsed_result['Localidad']))
        original_df.at[index,'COD_LOCALIDAD'] = get_cod_localidad(parsed_result['Localidad'])
        original_df.at[index,'SEC_CATASTRAL'] = parsed_result['Sector catastral']
        assign_upz(original_df,index,parsed_result['UPZ'])
        return "Rebuilt"
    else:
        return "Not processed"


In [ ]:
def rebuild_address_in_nuse(original_df, index):
    log_text = original_df.at[index,'LOG_TEXT']
    address_found = re.search(address_regex,log_text)

    if address_found != None:
        parsed_address = clean_address(address_found)
        print(parsed_address.strip())
        original_df.at[index,'STR_DIRECCION_INCIDENTE'] = parsed_address.strip()
        return "Rebuilt"
    else:
        original_df.at[index,'STR_DIRECCION_INCIDENTE'] = 'ND'
        return "Not processed"

def clean_address(address_found):
    exclude_char_list = ['~','/','*','(',')']
    one_occurrence = address_found.group().split(',,,')[0].replace(',',' ')
    final_address = one_occurrence
    
    for char in exclude_char_list:
        if char in one_occurrence:
            final_address = final_address.split(char)[0]
            
    numbers_in_substring = re.sub('[^0-9]','', final_address)
    numbers_proportion = len(numbers_in_substring)/len(final_address)
    
    if numbers_proportion < 0.2:
        final_address = 'ND'
    
    return final_address


## Implement rebuild methods

In [ ]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/merged_nuse.csv'
merged_nuse=pd.read_csv(data_location,delimiter=",")

In [ ]:
pd.DataFrame({"Tipo de dato":merged_nuse.dtypes.values,
              "Celdas con valor '-'":(merged_nuse == '-').sum().values,
              "Celdas con valor ''":(merged_nuse == '').sum().values,
              "Celdas con valor ' '":(merged_nuse == ' ').sum().values,
              "Celdas vacías": merged_nuse.isna().sum().values},
             index=merged_nuse.columns)

### Rebuild address through log_text

In [ ]:
#Try to rebuild missing address through log_text field
df_empty_locations_without_address = merged_nuse.loc[merged_nuse['STR_DIRECCION_INCIDENTE'] == '-']
list_idx_rebuild_address = list(df_empty_locations_without_address.index.values)

In [ ]:
len(list_idx_rebuild_address)

In [ ]:
address_regex= '(CL|DG|KR|TV)+\s\d+.*(,,)'
registers_to_process = len(list_idx_rebuild_address)
rebuilt_registers = 0
registers_not_processed = 0
other_condition_counter = 0

for index in list_idx_rebuild_address:
    state = rebuild_address_in_nuse(merged_nuse, index)
    
    if state == "Rebuilt":
        rebuilt_registers += 1
    elif state == "Not processed":
        registers_not_processed += 1
    else:
        other_condition_counter += 1
    
    print('Rebuilt registers: ',rebuilt_registers,'/',registers_to_process)
    print('Registers not processed: ',registers_not_processed, '/', registers_to_process)

In [ ]:
merged_nuse.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/rebuild_address_nuse_18112019.csv',index=None)

In [19]:
pd.DataFrame({"Tipo de dato":merged_nuse.dtypes.values,
              "Celdas con valor '-'":(merged_nuse == '-').sum().values,
              "Celdas con valor 'ND'":(merged_nuse == 'ND').sum().values,
              "Celdas vacías": merged_nuse.isna().sum().values},
             index=merged_nuse.columns)

,Tipo de dato,Celdas con valor '-',Celdas con valor 'ND',Celdas vacías
STR_NUMERO_INTERNO,object,0,0,0
FECHA,object,0,0,0
HORA,int64,0,0,0
ANIO,int64,0,0,0
MES,int64,0,0,0
COD_LOCALIDAD,int64,0,0,0
LOCALIDAD,object,0,0,0
COD_UPZ,object,2453,0,0
UPZ,object,2453,0,0
COD_SEC_CATAST,object,2453,0,0


### Rebuild location through address

In [28]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/rebuild_address_nuse_18112019.csv'
df_input = pd.read_csv(data_location,delimiter=",")

In [29]:
df1 = df_input.loc[df_input['COD_UPZ'] == '-']
df2 = df_input.loc[df_input['UPZ'] == '-']
df3 = df_input.loc[df_input['COD_SEC_CATAST'] == '-']
df4 = df_input.loc[df_input['SEC_CATASTRAL'] == '-']
df5 = df_input.loc[df_input['COD_BARRIO'] == '-']
df6 = df_input.loc[df_input['BARRIO'] == '-']

In [30]:
df1.equals(df2) and df1.equals(df3) and df1.equals(df4) and df1.equals(df5) and df1.equals(df6)

True

In [32]:
#Try to rebuild 'sector catastral', 'UPZ', 'localidad', 'latitud', 'longitud' through address
df_empty_locations_with_address = df1.loc[df1['STR_DIRECCION_INCIDENTE'] != 'ND']
list_idx_rebuild_location = list(df_empty_locations_with_address.index.values)
len(list_idx_rebuild_location)

1254

In [ ]:
#Rebuild 'sector catastral', 'UPZ', 'localidad', 'latitud', 'longitud' using web scraping
df_output = df_input
url='https://mapas.bogota.gov.co'
driver = ws_address.web_scrap_page(url)
registers_to_process = len(list_idx_rebuild_location)
rebuilt_registers = 0
registers_not_processed = 0
other_condition_counter = 0

for index in list_idx_rebuild_location:
    state = rebuild_location_in_nuse(df_output, index, driver)
    
    if state == "Rebuilt":
        rebuilt_registers += 1
    elif state == "Not processed":
        registers_not_processed += 1
    else:
        other_condition_counter += 1
    
    print('Rebuilt registers: ',rebuilt_registers,'/',registers_to_process)
    print('Registers not processed: ',registers_not_processed, '/', registers_to_process)

In [ ]:
print(rebuilt_registers)
print(registers_not_processed)
print(other_condition_counter)

In [ ]:
df_output.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/rebuild_locations_nuse_19112019.csv',index=None)

In [ ]:
pd.DataFrame({"Tipo de dato":merged_nuse.dtypes.values,
              "Celdas con valor '-'":(merged_nuse == '-').sum().values,
              "Celdas con valor 'ND'":(merged_nuse == 'ND').sum().values,
              "Celdas vacías": merged_nuse.isna().sum().values},
             index=merged_nuse.columns)

In [31]:
#Registers without address can not be rebuilt
df_empty_locations_without_address = df1.loc[df1['STR_DIRECCION_INCIDENTE'] == 'ND']
list_idx_not_rebuild = list(df_empty_locations_without_address.index.values)
len(list_idx_not_rebuild)

1199

In [ ]:
# TODO: assign ND to df_empty_locations_without_address on location fields